In [0]:
### Install pf requirements
!pip install promptflow promptflow-tools keyrings.alt

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached promptflow-0.1.0b8-py3-none-any.whl (1.3 MB)
  Using cached promptflow_tools-0.1.0b11-py3-none-any.whl (27 kB)
  Using cached keyrings.alt-5.0.0-py3-none-any.whl (18 kB)
  Using cached pyarrow-13.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (40.0 MB)
  Using cached keyring-24.2.0-py3-none-any.whl (37 kB)
  Using cached waitress-2.1.2-py3-none-any.whl (57 kB)
  Using cached Pillow-10.1.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.6 MB)
  Using cached strictyaml-1.7.3-py3-none-any.whl (123 kB)
  Using cached pydash-6.0.2-py3-none-any.whl (85 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Using cached ruamel.yaml-0.17.40-py3-none-any.whl (113 kB)
  Using cached opencensus_ext_azure-1.1.11-py2.py3-none-any.whl (43 kB)
  Using cached openai-0.27.10-py3-none-any.whl (76 kB)
  Using cached dataset-1.6.2-py2.p

In [0]:
dbutils.library.restartPython()

### Check to ensure promptflow is installed: !pf -v should display "0.1.0b8"

In [0]:
!pf -v

0.1.0b8


### Start by checking promptflow will run locally, setup connections defined in the flow.dag.yaml

In [0]:
#Create pf connection

from promptflow import PFClient
from promptflow.entities import AzureOpenAIConnection

# PFClient can help manage your runs and connections.
pf = PFClient()

try:
    conn_name = "aoai-connection"
    conn = pf.connections.get(name=conn_name)
    print("using existing connection")
except:
    #use keyvault to store secret keys
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="REPLACE_KEY",
        api_base="REPLACE_API_BASE",
        api_type="azure",
        api_version= "2023-07-01-preview",
    )

    conn = pf.connections.create_or_update(connection)
    print("successfully created connection")

print(conn)

successfully created connection
name: aoai-connection
module: promptflow.connections
created_date: '2023-11-09T05:08:52.212200'
last_modified_date: '2023-11-09T05:08:52.212200'
type: azure_open_ai
api_key: '******'
api_base: https://eu2-oai.openai.azure.com/
api_type: azure
api_version: 2023-07-01-preview



In [0]:
from promptflow import PFClient
import os

pf = PFClient()

flow_path =  "../chat_csv_model/promptflow"  # "web-classification" is the directory name
question = 'which states have the highest sales index?'
# Test flow
flow_inputs = {"columnlist": '''[ uid : identifier, geo : geography, naics_code_3 : industry code,
          naics_code_2 : industry code, period : date format YYYYMMDD,
          count_index : count_index, sales_index : sales_index, count_mom_diff :
          count_mom_diff, count_mom_rank : count_mom_rank, count_yoy_diff :
          count_yoy_diff, count_yoy_rank : count_yoy_rank,  sales_mom_diff :
          sales_mom_diff, sales_mom_rank : sales_mom_rank, sales_yoy_diff :
          sales_yoy_diff, sales_yoy_rank : sales_yoy_rank, count_index_adj :
          count_index_adj, sales_index_adj :
          sales_index_adj,  count_adj_mom_diff : count_adj_mom_diff,
          count_adj_mom_rank : count_adj_mom_rank, count_adj_yoy_diff :
          count_adj_yoy_diff, count_adj_yoy_rank : count_adj_yoy_rank,
          sales_adj_mom_diff : sales_adj_mom_diff, sales_adj_mom_rank :
          sales_adj_mom_rank, sales_adj_yoy_diff : sales_adj_yoy_diff,
          sales_adj_yoy_rank : sales_adj_yoy_rank, partition_by : YYYYMM ]''',
        'filename': 'Mock_Count_index_Data_20230928G.csv',
        'question': question }  # The inputs of the flow.
flow_result = pf.test(flow=flow_path, inputs=flow_inputs)
print(flow_result['answer'])

2023-11-09 05:08:55 +0000    1727 execution.flow     INFO     Start to run 6 nodes with concurrency level 16.
2023-11-09 05:08:55 +0000    1727 execution.flow     INFO     Executing node llm_python. node run id: 4ad0726b-fa66-405e-828d-403c132b47c0_llm_python_0
2023-11-09 05:08:56 +0000    1727 execution.flow     INFO     Node llm_python completes.
2023-11-09 05:08:56 +0000    1727 execution.flow     INFO     Executing node analyze_df. node run id: 4ad0726b-fa66-405e-828d-403c132b47c0_analyze_df_0
2023-11-09 05:08:56 +0000    1727 execution.flow     INFO     [analyze_df in line 0 (index starts from 0)] stdout> import pandas as pd
df = pd.read_csv('Mock_Count_index_Data_20230928G.csv')
# Assuming your dataframe is named 'df'
highest_sales_index_states = df.groupby('geo')['sales_index'].max().sort_values(ascending=False)
ans = highest_sales_index_states
print(ans)
2023-11-09 05:08:58 +0000    1727 execution.flow     INFO     [analyze_df in line 0 (index starts from 0)] stdout> CompletedP

### Create Model Class to register as promptflow model.  Please note the promptflow expects model "gpt-35-turbo" to be deployed to endpoint

###TODO: Replace connection key and api_base

In [0]:
import mlflow
import mlflow.pyfunc
from mlflow.models import infer_signature

# Define the model class
class chatcsvpf(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        from promptflow import PFClient
        from promptflow.entities import AzureOpenAIConnection
        import pandas as pd
        import numpy as np

        # PFClient can help manage your runs and connections.
        pf = PFClient()

        try:
            conn_name = "aoai-connection"
            conn = pf.connections.get(name=conn_name)
            print("using existing connection")
        except:
            #use keyvault to store secret keys
            connection = AzureOpenAIConnection(
                name=conn_name,
                api_key="REPLACE_KEY",
                api_base="REPLACE_API_BASE",
                api_type="azure",
                api_version= "2023-07-01-preview",
            )
            conn = pf.connections.create_or_update(connection)      
        self.flow_path = "{}/promptflow".format(context.artifacts["chatcsvpf"])#"promptflow"  # "web-classification" is the directory name  


    def predict(self, context, model_input, params=None):
        import pandas as pd
        import numpy as np
        pf = PFClient()
        question = model_input["question"][0]
        if isinstance(question, np.ndarray):
            question = question.item()
        cols = model_input["columnlist"][0]
        if isinstance(cols, np.ndarray):
            cols = cols.item()
        filename = model_input["filename"][0]
        if isinstance(filename, np.ndarray):
            filename = filename.item()
        # Test flow
        flow_inputs = {"columnlist": cols,
                        'filename': filename,
                'question': question }  # The inputs of the flow.
        flow_result = pf.test(flow=flow_path, inputs=flow_inputs)
        return(flow_result)


### Register promptflow model to mlflow model registry

In [0]:
import os
import mlflow

with mlflow.start_run() as run:
    artifacts = {"chatcsvpf": "../chat_csv_model"}
    modelpath = "chat_csv-promptflow"
    sample_input = {"columnlist": '''[ uid : identifier, geo : geography, naics_code_3 : industry code,
          naics_code_2 : industry code, period : date format YYYYMMDD,
          count_index : count_index, sales_index : sales_index, count_mom_diff :
          count_mom_diff, count_mom_rank : count_mom_rank, count_yoy_diff :
          count_yoy_diff, count_yoy_rank : count_yoy_rank,  sales_mom_diff :
          sales_mom_diff, sales_mom_rank : sales_mom_rank, sales_yoy_diff :
          sales_yoy_diff, sales_yoy_rank : sales_yoy_rank, count_index_adj :
          count_index_adj, sales_index_adj :
          sales_index_adj,  count_adj_mom_diff : count_adj_mom_diff,
          count_adj_mom_rank : count_adj_mom_rank, count_adj_yoy_diff :
          count_adj_yoy_diff, count_adj_yoy_rank : count_adj_yoy_rank,
          sales_adj_mom_diff : sales_adj_mom_diff, sales_adj_mom_rank :
          sales_adj_mom_rank, sales_adj_yoy_diff : sales_adj_yoy_diff,
          sales_adj_yoy_rank : sales_adj_yoy_rank, partition_by : YYYYMM ]''',
        'filename': 'Mock_Count_index_Data_20230928G.csv',
        'question': "question" }  # The inputs of the flow.
    mlflow.pyfunc.log_model(
        modelpath,
        python_model=chatcsvpf(),
        artifacts=artifacts,
        signature=infer_signature(sample_input, ["Run"]),
        extra_pip_requirements=["promptflow", "pandas", "promptflow-tools", "keyrings.alt", "numpy"]
    )
    run_id = run.info.run_id
    mlflow.register_model(f"runs:/{run_id}/{modelpath}", modelpath)
    

2023/11/09 05:10:29 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/16 [00:00<?, ?it/s]

2023/11/09 05:10:36 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
Successfully registered model 'chat_csv-promptflow'.
2023/11/09 05:10:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: chat_csv-promptflow, version 1
Created version '1' of model 'chat_csv-promptflow'.


# Go to Model Registry to confirm model has been created